In [1]:
# Se importan las librerias a utilizar
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

In [2]:
# se carga el dataset
df  = pd.read_csv('./dataset/movies_limpio.csv', delimiter=',')
# se selecciona las columnas de interés
columnas_seleccionadas = ['overview', 'popularity', 'tagline', 'title']  # Reemplaza con los nombres de las columnas que deseas seleccionar
df = df[columnas_seleccionadas]
df.sample(10)

C:\Users\HP\AppData\Local\Temp\ipykernel_14940\2947862856.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df  = pd.read_csv('./dataset/movies_limpio.csv', delimiter=',')


,overview,popularity,tagline,title
23058,Vassili is an aged prostitute with killer inst...,1.073478,NaN,Our Paradise
23036,Satché is about to die. He decides to make his...,0.998359,NaN,Today
8139,Sylvia Stickles runs a convenience store with ...,7.665431,Threatening the very limits of common decency.,A Dirty Shame
4222,A skilled young hockey prospect hoping to attr...,4.204661,The ice... The fire... The fight... To be the ...,Youngblood
36604,The film evokes a childhood in rural Lithuania...,0.473698,NaN,The Issa Valley
28972,Detectives and best friends Andreas and Simon ...,3.042462,How far would you go?,A Second Chance
40421,"Shy, a small-town butch with a nagging messiah...",0.05154,NaN,By Hook or by Crook
5815,Eliza and Debbie are two sisters who don't alw...,5.341749,You don't need extraordinary powers to do extr...,The Wild Thornberrys Movie
5750,"Looking to escape her unhappy marriage, villai...",4.187079,She wants it all...,The Last Seduction
25631,Maurizio Merli stars as a hatchet-wielding bou...,2.114454,NaN,A Man Called Blade


In [3]:
# Se eliminan los registros duplicados
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [4]:
#se imputan los nulos o nan con ''
titulo = df['title'].fillna('')
resumen = df['overview'].fillna('')
frase   = df['tagline'].fillna('')

In [5]:
tfidf = TfidfVectorizer(stop_words='english', lowercase=True) # configuracion de TfidfVectorizery, en idioma ingles

In [6]:
# se crea la una matriz TF-IDF con las puntaciones del campo title
tfidf_matrix = tfidf.fit_transform(titulo)

In [7]:
print(tfidf_matrix)

  (0, 19260)	0.5339365409301036
  (0, 20542)	0.8455245533156301
  (1, 10646)	1.0
  (2, 12911)	0.4385911829358217
  (2, 14434)	0.4950577534438312
  (2, 8663)	0.7500370624216536
  (3, 6866)	0.7982840579051067
  (3, 21743)	0.6022811327731894
  (4, 9820)	0.506957062721474
  (4, 2927)	0.6143944491204045
  (4, 7111)	0.6045775363399223
  (5, 9141)	1.0
  (6, 17314)	1.0
  (7, 9632)	0.8034553065631692
  (7, 20410)	0.5953650731740011
  (8, 5290)	0.5383829329153123
  (8, 19437)	0.8427003130090235
  (9, 8404)	1.0
  (10, 15835)	0.8041581820460744
  (10, 886)	0.5944153583550419
  (11, 12048)	0.6667325388456612
  (11, 5265)	0.4474140827223685
  (11, 6047)	0.5960607017964868
  (12, 1784)	1.0
  (13, 14129)	1.0
  :	:
  (45352, 12669)	0.47019824252249104
  (45353, 5360)	0.6886920741841239
  (45353, 9136)	0.7250539476176715
  (45354, 13331)	1.0
  (45355, 13480)	1.0
  (45356, 17095)	0.6666752269608563
  (45356, 13024)	0.5325537889656329
  (45356, 19034)	0.5214696574250882
  (45357, 9553)	0.8374075465717441


In [8]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix) # se calcula la matriz de coseno para los vectores.

In [10]:
def get_recommendations(df, column, value, cosine_similarities, limit=5):
    """ Retorna un dataframe de 5 registros recomendados según la 
        Frecuencia de término Frecuencia de documento inversa (TF-IDF) 
        y la similitud del coseno.
    
    Args:
        df (object): dataframe que contiene el texto a analizar. 
        column (string): nombre de la columna utilizada. 
        value (string): nombre del titulo a tener las recomendaciones.
        cosine_similarities (array): matriz de similitud del cosenol
        limit (int, optional): retorna la cantidad de registros a retornar recomendados. 
        
    Returns: 
        Pandas dataframe. 
    """
    
    # Return indices for the target dataframe column and drop any duplicates
    indices = pd.Series(df.index, index=df[column]).drop_duplicates()

    # Get the index for the target value
    target_index = indices[value]

    # Get the cosine similarity scores for the target value
    cosine_similarity_scores = list(enumerate(cosine_similarities[target_index]))

    # Sort the cosine similarities in order of closest similarity
    cosine_similarity_scores = sorted(cosine_similarity_scores, key=lambda x: x[1], reverse=True)

    # Return tuple of the requested closest scores excluding the target item and index
    cosine_similarity_scores = cosine_similarity_scores[1:limit+1]

    # Extract the tuple values
    index = (x[0] for x in cosine_similarity_scores)
    scores = (x[1] for x in cosine_similarity_scores)    

    # Get the indices for the closest items
    recommendation_indices = [i[0] for i in cosine_similarity_scores]

    # Get the actutal recommendations
    recommendations = df[column].iloc[recommendation_indices]

    # Return a dataframe
    df = pd.DataFrame(list(zip(index, recommendations, scores)), 
                      columns=['index','recommendation', 'cosine_similarity_score']) 

    return df

In [11]:
recommendations = get_recommendations(df, 
                                      'title', 
                                      'Toy Story', 
                                      cosine_similarities)
recommendations.head(5)

,index,recommendation,cosine_similarity_score
0,2994,Toy Story 2,1.000000
1,15337,Toy Story 3,1.000000
2,4794,The Toy,0.845525
3,16142,The Toy,0.845525
4,21898,Toy Story of Terror!,0.817672


In [13]:
def obtenerListaPeliculas(pelicula):
    recommendations = get_recommendations(df, 
                                      'title', 
                                      pelicula, 
                                      cosine_similarities)
    return recommendations['recommendation']
    
print(obtenerListaPeliculas('Toy Story'))

0             Toy Story 2
1             Toy Story 3
2                 The Toy
3                 The Toy
4    Toy Story of Terror!
Name: recommendation, dtype: object
